In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import random
import plotly.express as px

In [2]:
df = pd.read_parquet("Shopping Carts.parquet")

In [3]:
df.reset_index(inplace=True)

In [16]:
df

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador


In [43]:
df_1 = df.groupby(["order_id","user_id"]).agg(count = ("user_id","nunique")).reset_index()

In [46]:
df_1["count"].unique()

array([1], dtype=int64)

: 

Inputs = order-Größe;wochentag;bei welchem Anteil der bisherigen Bestellungen dieses users wurde Trinkgeld gegeben;county;Zusammensetzung dieser Bestellung früherer BEstellungen

Ziel: Vorhersage, ob ein User bei einer gerade aufgegeben Bestellung Trinkgeld geben wird

In [9]:
df.describe()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,aisle_id,department_id
count,6.132669e+06,6.132669e+06,6.132669e+06,6.132669e+06,6.132669e+06,6.132669e+06,6.132669e+06,6.132669e+06,5.757096e+06,6.132669e+06,6.132669e+06,6.132669e+06
mean,3.034141e+05,2.555715e+04,8.347707e+00,5.891352e-01,1.034604e+05,1.708789e+01,2.736021e+00,1.343908e+01,1.138992e+01,4.474964e-01,7.137251e+01,9.941165e+00
std,1.751674e+05,1.409440e+04,7.132200e+00,4.919908e-01,5.942233e+04,1.749323e+01,2.097220e+00,4.256280e+00,8.934196e+00,4.972358e-01,3.817693e+01,6.286584e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,1.517370e+05,1.351200e+04,3.000000e+00,0.000000e+00,5.243000e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00
50%,3.032450e+05,2.523000e+04,6.000000e+00,1.000000e+00,1.032530e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,0.000000e+00,8.300000e+01,9.000000e+00
75%,4.551400e+05,3.786600e+04,1.100000e+01,1.000000e+00,1.548280e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01,1.000000e+00,1.070000e+02,1.600000e+01
max,6.069150e+05,4.968800e+04,1.450000e+02,1.000000e+00,2.062010e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01,1.000000e+00,1.340000e+02,2.100000e+01


In [11]:
df.county.unique()

array(['Inyo', 'Calaveras', 'Monterey', 'Glenn', 'Plumas', 'Sacramento',
       'Santa Clara', 'Marin', 'San Francisco', 'San Luis Obispo',
       'San Mateo', 'Tuolumne', 'Mariposa', 'Los Angeles', 'Imperial',
       'Santa Barbara', 'Alpine', 'Fresno', 'Sutter', 'Orange',
       'Humboldt', 'Yuba', 'Kern', 'Mendocino', 'San Benito', 'Ventura',
       'Tulare', 'Modoc', 'Merced', 'San Diego', 'Siskiyou', 'Nevada',
       'Stanislaus', 'Amador', 'Contra Costa', 'Shasta', 'Riverside',
       'Santa Cruz', 'Sonoma', 'Madera', 'Lake', 'Del Norte', 'El Dorado',
       'Sierra', 'Yolo', 'Colusa', 'Alameda', 'Solano', 'Kings', 'Mono',
       'Napa', 'Lassen', 'San Joaquin', 'Tehama', 'Trinity', 'Placer',
       'San Bernardino'], dtype=object)

In [13]:
df.aisle.unique()

array(['fresh fruits', 'other creams cheeses', 'fresh vegetables',
       'packaged cheese', 'canned meat seafood', 'yogurt',
       'air fresheners candles', 'laundry', 'refrigerated', 'coffee',
       'tofu meat alternatives', 'ice cream ice',
       'frozen appetizers sides', 'lunch meat', 'soy lactosefree',
       'butter', 'trash bags liners', 'cookies cakes', 'oral hygiene',
       'dog food care', 'bread', 'poultry counter', 'frozen produce',
       'spices seasonings', 'grains rice dried goods',
       'water seltzer sparkling water', 'paper goods', 'frozen juice',
       'milk', 'baby food formula', 'breakfast bakery',
       'packaged produce', 'packaged vegetables fruits',
       'hot dogs bacon sausage', 'condiments', 'eggs',
       'soup broth bouillon', 'pickled goods olives',
       'fresh dips tapenades', 'chips pretzels',
       'bulk grains rice dried goods', 'mint gum', 'frozen pizza',
       'frozen dessert', 'missing', 'canned jarred vegetables',
       'nuts seeds

In [ ]:
df.isna().any()

In [ ]:
df.dtypes

In [ ]:
df.order_dow

In [ ]:
df[df.days_since_prior_order.isna()]

In [4]:
ind = df.groupby("user_id")[["index"]].min().reset_index()["index"]

In [5]:
px.scatter(df.groupby("days_since_prior_order")["index"].count())

In [6]:
df_1 = df.loc[df.index.isin(ind)]

In [7]:
df_1

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
231,4415176,21,40310,21,1,86865,20,0,9,17.0,0,Oven Roasted Turkey Breast,96,20,deli,lunch meat,Alpine
918,3379043,91,41570,4,0,203870,25,2,9,3.0,0,Active Dry Yeast,17,13,pantry,baking ingredients,Calaveras
1429,2878013,136,27104,9,1,139365,45,6,14,3.0,0,Fresh Cauliflower,83,4,produce,fresh vegetables,Imperial
2111,3123253,199,7948,7,0,78359,17,3,6,5.0,1,Organic Unsalted Butter,36,16,dairy eggs,butter,Fresno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132633,4757328,606912,21938,7,0,206030,31,2,9,5.0,1,Green Bell Pepper,83,4,produce,fresh vegetables,Sutter
6132639,4003189,606913,29370,9,1,42599,10,6,20,30.0,0,Organic White Cheddar Popcorn,23,19,snacks,popcorn jerky,Marin
6132648,4003182,606913,5876,12,1,42599,10,6,20,30.0,0,Organic Lemon,24,4,produce,fresh fruits,Marin
6132658,2493336,606914,48101,1,0,193225,18,5,13,4.0,1,Poultry Magic Seasoning Blends,104,13,pantry,spices seasonings,Calaveras


In [8]:
px.scatter(df_1.groupby("days_since_prior_order")["index"].count())

In [9]:
px.scatter(df.groupby("order_dow")["index"].count())

In [10]:
px.scatter(df.groupby("order_hour_of_day")["index"].count())

In [11]:
px.scatter(df.groupby("department")["index"].count())

In [12]:
px.scatter(df.groupby("aisle")["index"].count())

In [13]:
px.scatter(df.groupby("county")["index"].count())

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
# fig = px.choropleth(df, geojson=counties, locations='county', 
#                            scope="usa",
#                            featureidkey="properties.NAME"
                           
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()